# Notebook Contents

- [Imports](#Imports)
- [Data](#Data)
- [Data Cleaning](#Data-Cleaning)
- [Preprocessing](#Preprocessing)
    - [Multicolinearity - VIF](#Multicolinearity---VIF)
- [Features](#Features)
- [Random Forest Modeling](#Random-Forest-Modeling)
    - [4-Seam](#Linear-Regression---4-Seam)
    - [Cutter](#Linear-Regression---Cutter)
    - [Sinker](#Linear-Regression---Sinker)
    - [Slider](#Linear-Regression---Slider)
    - [Curveball](#Linear-Regression---Curveball)
    - [Changeup](#Linear-Regression---Changeup)

# Imports

In [1]:
from pprint import pprint
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

# Data

In [2]:
data = pd.read_csv('../data/model-pitches.csv', index_col = [0])
data.dropna(subset = ['pitch_type', 'velo', 'spin_rate', 'pfx_-x', 
                      'release_extension', 'delta_run_exp'], inplace = True)

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

re = pd.read_csv('../data/run_expectancy_table.csv', index_col = [0])
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])

data['re_end_state'] = data['delta_run_exp'] + data['re']
data['re24'] = data['re_end_state'] - data['re'] + data['runs']

pd.set_option('max_columns', None)
print(data.shape)
data.head(5)

(705403, 52)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,delta_run_exp,stand,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,home_runs,away_runs,runs,re,re_end_state,re24
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,-0.073,R,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.025,-0.073
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,-0.027,R,foul,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.071,-0.027
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,-0.020,R,foul,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.078,-0.020
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0.016,R,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.114,0.016
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,-0.189,L,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,0,0,0,0.254,0.065,-0.189


# Preprocessing

### Multicolinearity - VIF
**Independent Variables:** Velocity, Spin Rate, VB, HB, Release Extension, Horizontal Release Position, Vertical Release Position, Horizontal Plate Coords, Vertical Plate Coords

**Dependent Variable:** re24


In [3]:
features = data[['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 'release_extension', 
                 'release_pos_x', 'release_pos_z', 're24',
                 'pitch_type', 'p_throws', 'stand']]
# 'plate_x', 'plate_z',
features_vif = features.select_dtypes([np.number])
vif_data = pd.DataFrame()
vif_data["feature"] = features_vif.columns

vif_data["VIF"] = [variance_inflation_factor(features_vif.values, i)
                   for i in range(len(features_vif.columns))]

vif_data.sort_values(by = 'VIF').head(10)

,feature,VIF
7,re24,1.005335
2,pfx_-x,1.374450
5,release_pos_x,1.434496
3,pfx_z,2.960615
1,spin_rate,50.591612
6,release_pos_z,103.225086
4,release_extension,152.278243
0,velo,273.199817


# Features

In [4]:
zero_outs = data.loc[data['outs_when_up'] == 0]
print('0 outs:', zero_outs.shape)
one_out = data.loc[data['outs_when_up'] == 1]
print('1 out:', one_out.shape)
two_outs = data.loc[data['outs_when_up'] == 2]
print('2 outs:', two_outs.shape)

0 outs: (244451, 52)
1 out: (232568, 52)
2 outs: (228384, 52)


In [5]:
ff = features.loc[features['pitch_type'] == 'FF']
fc = features.loc[features['pitch_type'] == 'FC']
fastball = ff.append(fc)
si = features.loc[features['pitch_type'] == 'SI']
fastball = fastball.append(si)
print('Fastball shape:', fastball.shape)
sl = features.loc[features['pitch_type'] == 'SL']
cu = features.loc[features['pitch_type'] == 'CU']
breaking_ball = sl.append(cu)
kc = features.loc[features['pitch_type'] == 'KC']
breaking_ball = breaking_ball.append(kc)
print('Breaking Ball:', breaking_ball.shape)
ch = features.loc[features['pitch_type'] == 'CH']
fs = features.loc[features['pitch_type'] == 'FS']
offspeed = ch.append(fs)
print('Off speed shape:', offspeed.shape)
rhp = features.loc[features['p_throws'] == 'R']
print('RHP shape:', rhp.shape)
lhp = features.loc[features['p_throws'] == 'L']
print('LHP shape:', lhp.shape)
rhp_rhh = features.loc[(features['p_throws'] == 'R') & (features['stand'] == 'R')]
print('RHP & RHH shape:', rhp_rhh.shape)
rhp_lhh = features.loc[(features['p_throws'] == 'R') & (features['stand'] == 'L')]
print('RHP & LHH shape:', rhp_lhh.shape)
lhp_rhh = features.loc[(features['p_throws'] == 'L') & (features['stand'] == 'R')]
print('LHP & RHH shape:', lhp_rhh.shape)
lhp_lhh = features.loc[(features['p_throws'] == 'L') & (features['stand'] == 'L')]
print('LHP & LHH shape:', lhp_lhh.shape)
rhp_fastball = fastball.loc[fastball['p_throws'] == 'R']
print('RHP Fastball shape:', rhp_fastball.shape)
lhp_fastball = fastball.loc[fastball['p_throws'] == 'L']
print('LHP Fastball shape:', lhp_fastball.shape)
rhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'R']
print('RHP Breaking Ball shape:', rhp_breaking_ball.shape)
lhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'L']
print('LHP Breaking Ball shape:', lhp_breaking_ball.shape)
rhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'R']
print('RHP Offspeed shape:', rhp_offspeed.shape)
lhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'L']
print('LHP Offspeed shape:', lhp_offspeed.shape)

Fastball shape: (406259, 11)
Breaking Ball: (207982, 11)
Off speed shape: (91162, 11)
RHP shape: (496498, 11)
LHP shape: (208905, 11)
RHP & RHH shape: (267548, 11)
RHP & LHH shape: (228950, 11)
LHP & RHH shape: (149824, 11)
LHP & LHH shape: (59081, 11)
RHP Fastball shape: (283224, 11)
LHP Fastball shape: (123035, 11)
RHP Breaking Ball shape: (152383, 11)
LHP Breaking Ball shape: (55599, 11)
RHP Offspeed shape: (60891, 11)
LHP Offspeed shape: (30271, 11)


# Linear Regression Model

## Fastball RHP

#### 4-Seam, Cutter, Sinker

In [6]:
features_fastball_r = rhp_fastball.select_dtypes([np.number])
X = features_fastball_r.drop(columns = ['re24'])
X = sm.add_constant(X)
y = features_fastball_r['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_fastball_r = sm.OLS(y_train, X_train).fit()
pred_fastball_r = ols_fastball_r.predict(X_test)
fitted_vals_fastball_r = ols_fastball_r.fittedvalues
residuals_fastball_r = ols_fastball_r.resid

print('MSE:', round(metrics.mean_squared_error(y_test, pred_fastball_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, pred_fastball_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, pred_fastball_r), 4))
print(ols_fastball_r.summary())

MSE: 0.1833
RMSE: 0.4281
MAE: 0.1629
                            OLS Regression Results                            
Dep. Variable:                   re24   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     12.29
Date:                Sun, 06 Mar 2022   Prob (F-statistic):           8.10e-16
Time:                        22:05:33   Log-Likelihood:            -1.2339e+05
No. Observations:              212418   AIC:                         2.468e+05
Df Residuals:                  212410   BIC:                         2.469e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
c

## Fastball LHP

#### 4-Seam, Cutter, Sinker

In [7]:
features_fastball_l = lhp_fastball.select_dtypes([np.number])
X = features_fastball_l.drop(columns = ['re24'])
X = sm.add_constant(X)
y = features_fastball_l['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_fastball_l = sm.OLS(y_train, X_train).fit()
pred_fastball_l = ols_fastball_l.predict(X_test)
fitted_vals_fastball_l = ols_fastball_l.fittedvalues
residuals_fastball_l = ols_fastball_l.resid

print('MSE:', round(metrics.mean_squared_error(y_test, pred_fastball_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, pred_fastball_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, pred_fastball_l), 4))
print(ols_fastball_l.summary())

MSE: 0.1866
RMSE: 0.432
MAE: 0.1644
                            OLS Regression Results                            
Dep. Variable:                   re24   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.785
Date:                Sun, 06 Mar 2022   Prob (F-statistic):             0.0854
Time:                        22:05:33   Log-Likelihood:                -50069.
No. Observations:               92276   AIC:                         1.002e+05
Df Residuals:                   92268   BIC:                         1.002e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
co

## Breaking Ball RHP

#### Slider, Curveball, Knuckle Curve

In [8]:
features_bb_r = rhp_breaking_ball.select_dtypes([np.number])
X = features_bb_r.drop(columns = ['re24'])
X = sm.add_constant(X)
y = features_bb_r['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_bb_r = sm.OLS(y_train, X_train).fit()
pred_bb_r = ols_bb_r.predict(X_test)
fitted_vals_bb_r = ols_bb_r.fittedvalues
residuals_bb_r = ols_bb_r.resid

print('MSE:', round(metrics.mean_squared_error(y_test, pred_bb_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, pred_bb_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, pred_bb_r), 4))
print(ols_bb_r.summary())

MSE: 0.1761
RMSE: 0.4196
MAE: 0.1533
                            OLS Regression Results                            
Dep. Variable:                   re24   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     7.384
Date:                Sun, 06 Mar 2022   Prob (F-statistic):           6.75e-09
Time:                        22:05:34   Log-Likelihood:                -62551.
No. Observations:              114287   AIC:                         1.251e+05
Df Residuals:                  114279   BIC:                         1.252e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
c

## Breaking Ball LHP

#### Slider, Curveball, Knuckle Curve

In [9]:
features_bb_l = lhp_breaking_ball.select_dtypes([np.number])
X = features_bb_l.drop(columns = ['re24'])
X = sm.add_constant(X)
y = features_bb_l['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_bb_l = sm.OLS(y_train, X_train).fit()
pred_bb_l = ols_bb_l.predict(X_test)
fitted_vals_bb_l = ols_bb_l.fittedvalues
residuals_bb_l = ols_bb_l.resid

print('MSE:', round(metrics.mean_squared_error(y_test, pred_bb_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, pred_bb_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, pred_bb_l), 4))
print(ols_bb_l.summary())

MSE: 0.182
RMSE: 0.4266
MAE: 0.1535
                            OLS Regression Results                            
Dep. Variable:                   re24   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.501
Date:                Sun, 06 Mar 2022   Prob (F-statistic):           2.44e-06
Time:                        22:05:34   Log-Likelihood:                -22565.
No. Observations:               41699   AIC:                         4.515e+04
Df Residuals:                   41691   BIC:                         4.521e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
co

## Offspeed RHP

#### Changeup, Splitter

In [10]:
features_os_r = rhp_offspeed.select_dtypes([np.number])
X = features_os_r.drop(columns = ['re24'])
X = sm.add_constant(X)
y = features_os_r['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_os_r = sm.OLS(y_train, X_train).fit()
pred_os_r = ols_os_r.predict(X_test)
fitted_vals_os_r = ols_os_r.fittedvalues
residuals_os_r = ols_os_r.resid

print('MSE:', round(metrics.mean_squared_error(y_test, pred_os_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, pred_os_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, pred_os_r), 4))
print(ols_os_r.summary())

MSE: 0.1765
RMSE: 0.4201
MAE: 0.1646
                            OLS Regression Results                            
Dep. Variable:                   re24   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.244
Date:                Sun, 06 Mar 2022   Prob (F-statistic):             0.0280
Time:                        22:05:34   Log-Likelihood:                -25379.
No. Observations:               45668   AIC:                         5.077e+04
Df Residuals:                   45660   BIC:                         5.084e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
c

## Offspeed LHP

#### Changeup, Splitter

In [11]:
features_os_l = lhp_offspeed.select_dtypes([np.number])
X = features_os_l.drop(columns = ['re24'])
X = sm.add_constant(X)
y = features_os_l['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_os_l = sm.OLS(y_train, X_train).fit()
pred_os_l = ols_os_l.predict(X_test)
fitted_vals_os_l = ols_os_l.fittedvalues
residuals_os_l = ols_os_l.resid

print('MSE:', round(metrics.mean_squared_error(y_test, pred_os_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, pred_os_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, pred_os_l), 4))
print(ols_os_l.summary())

MSE: 0.2128
RMSE: 0.4613
MAE: 0.1733
                            OLS Regression Results                            
Dep. Variable:                   re24   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.521
Date:                Sun, 06 Mar 2022   Prob (F-statistic):             0.0137
Time:                        22:05:34   Log-Likelihood:                -12970.
No. Observations:               22703   AIC:                         2.596e+04
Df Residuals:                   22695   BIC:                         2.602e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
c

# Random Forest Model

## Fastball RHP

#### 4-Seam, Cutter, Sinker

In [17]:
# params = {
#    'n_estimators': [25, 75],
#    'max_depth': [None, 5, 6, 7, 8],
#    'min_samples_split': [1, 2, 4],
#    'min_samples_leaf': [2, 5, 10],
#    'n_jobs': [-1]
#}
    
# gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
# gs.fit(X_train, y_train)
# fastball_pred_r = gs.predict(X_test)

# print('Cross val score:', round((gs.best_score_), 4))

# gs.best_params_

In [19]:
features_fastball_r = rhp_fastball.select_dtypes([np.number])
X = features_fastball_r.drop(columns = ['re24'])
y = features_fastball_r['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y)

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
fastball_pred_r = rf.predict(X_test)

print('MSE:', round(metrics.mean_squared_error(y_test, fastball_pred_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, fastball_pred_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, fastball_pred_r), 4))

In [ ]:
print('Cross val score:', round(cross_val_score(rf, X_train, y_train).mean(), 4))

## Fastball LHP

#### 4-Seam, Cutter, Sinker

In [ ]:
features_fastball_l = lhp_fastball.select_dtypes([np.number])
X = features_fastball_l.drop(columns = ['re24'])
y = features_fastball_l['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
fastball_pred_l = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, fastball_pred_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, fastball_pred_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, fastball_pred_l), 4))
gs.best_params_

## Breaking Ball RHP

#### Slider, Curveball, Knuckle Curve

In [ ]:
features_bb_r = rhp_breaking_ball.select_dtypes([np.number])
X = features_bb_r.drop(columns = ['re24'])
y = features_bb_r['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
bb_pred_r = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, bb_pred_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, bb_pred_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, bb_pred_r), 4))
gs.best_params_

## Breaking Ball LHP

#### Slider, Curveball, Knuckle Curve

In [ ]:
features_bb_l = lhp_breaking_ball.select_dtypes([np.number])
X = features_bb_l.drop(columns = ['re24'])
y = features_bb_l['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
bb_pred_l = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, bb_pred_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, bb_pred_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, bb_pred_l), 4))
gs.best_params_

## Offspeed RHP

#### Changeup, Splitter

In [ ]:
features_os_r = rhp_offspeed.select_dtypes([np.number])
X = features_os_r.drop(columns = ['re24'])
y = features_os_r['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
os_pred_r = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, os_pred_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, os_pred_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, os_pred_r), 4))
gs.best_params_

## Offspeed LHP

#### Changeup, Splitter

In [ ]:
features_os_l = lhp_offspeed.select_dtypes([np.number])
X = features_os_l.drop(columns = ['re24'])
y = features_os_l['re24']

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'n_estimators': [25, 75],
    'max_depth': [None, 5, 6, 7, 8],
    'min_samples_split': [1, 2, 4],
    'min_samples_leaf': [2, 5, 10],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 10), param_grid = params)
gs.fit(X_train, y_train)
os_pred_l = gs.predict(X_test)

print('Cross val score:', round((gs.best_score_), 4))
print('MSE:', round(metrics.mean_squared_error(y_test, os_pred_l), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y_test, os_pred_l)), 4))
print('MAE:', round(metrics.mean_absolute_error(y_test, os_pred_l), 4))
gs.best_params_